In [3]:
from utils.query_utils import athena_query_to_df
import json
import re
import pandas as pd

In [4]:
games_df_query = """select "date", white, black, "result", 1 as count,
case
	when white = 'luckleland' and result = '1-0' then 1
	when black = 'luckleland' and result = '0-1' then 1
	else 0 
end as "win",
case
	when black = 'luckleland' and result = '1-0' then 1
	when white = 'luckleland' and result = '0-1' then 1
	else 0
end as "loss",
case
	when result = '1/2-1/2' then 1
	else 0
end as "draw",
gamestring, id_key
from lichess.lichess_api_data
order by date desc"""

In [5]:
games_df = athena_query_to_df(games_df_query)

In [6]:
games_df['id_key'] = games_df['id_key'].astype(str)

In [18]:
games_df.head()

,date,white,black,result,count,win,loss,draw,gamestring,id_key
0,2023-11-07,luckleland,hossein13601360,1-0,1,1,0,0,1. d4 d5 2. c4 e6 3. Nc3 Nf6 4. Bg5 Bb4 5. Qb3...,20231107163809
1,2023-11-07,luckleland,saNsaN200,0-1,1,0,1,0,1. d4 e5 2. c4 exd4 3. Qxd4 Nc6 4. Qe3+ Be7 5....,20231107152033
2,2023-11-07,Andrey_1983,luckleland,1/2-1/2,1,0,0,1,1. e4 d5 2. exd5 Nf6 3. Nc3 Nxd5 4. Nxd5 Qxd5 ...,20231107234133
3,2023-11-07,luckleland,antoniocbr,1-0,1,1,0,0,1. d4 d5 2. c4 dxc4 3. Nc3 e5 4. d5 Bb4 5. Qa4...,20231107221023
4,2023-11-07,Sharcman,luckleland,0-1,1,1,0,0,1. e4 d5 2. exd5 Nf6 3. c4 c6 4. dxc6 Nxc6 5. ...,20231107180926


In [22]:
def pgn_parser(row):
    game_string = row["gamestring"]
    id_key = row["id_key"]
    last_move = False
    move_number = 1
    game_dicts = []
    while not last_move:
        if move_number >= 20:
            last_move = True
        move_string = str(move_number) + "."
        next_move_string = str(move_number + 1) + "."
        running_game_str = game_string[:game_string.index(move_string) + len(move_string)]
        if next_move_string not in game_string:
            last_move = True
            move_list = game_string[game_string.index(move_string) + len(move_string):].split()
            game_dicts.append({"id_key": id_key, "move_number": move_string[:-1] + "w", "pgn_string": running_game_str + move_list[0]})
            if move_list[1] != '0-1' and move_list[1] != '1-0' and move_list[1] != '1/2-1/2':
                game_dicts.append({"id_key": id_key, "move_number":move_string[:-1] + "b", "pgn_string": running_game_str + " ".join(move_list[:-1])})
        else:
            move_list = game_string[game_string.index(move_string) + len(move_string):game_string.index(next_move_string)].split()  
            game_dicts.append({"id_key": id_key, "move_number":move_string[:-1] + "w", "pgn_string": running_game_str + move_list[0]})
            game_dicts.append({"id_key": id_key, "move_number":move_string[:-1] + "b", "pgn_string": running_game_str + " ".join(move_list)})
            move_number += 1
    return pd.DataFrame(game_dicts)

In [23]:
records = pgn_parser(test)

TypeError: 'RangeIndex' object is not callable

In [21]:
records

,id_key,move_number,pgn_string
0,0 20231107163809 1 20231107152033 2 2...,1w,0 1.d4
1,0 20231107163809 1 20231107152033 2 2...,1b,0 1.d4 d5
2,0 20231107163809 1 20231107152033 2 2...,2w,0 1. d4 d5 2.c4
3,0 20231107163809 1 20231107152033 2 2...,2b,0 1. d4 d5 2.c4 e6
4,0 20231107163809 1 20231107152033 2 2...,3w,0 1. d4 d5 2. c4 e6 3.Nc3
5,0 20231107163809 1 20231107152033 2 2...,3b,0 1. d4 d5 2. c4 e6 3.Nc3 Nf6
6,0 20231107163809 1 20231107152033 2 2...,4w,0 1. d4 d5 2. c4 e6 3. Nc3 Nf6 4.Bg5
7,0 20231107163809 1 20231107152033 2 2...,4b,0 1. d4 d5 2. c4 e6 3. Nc3 Nf6 4.Bg5 Bb4
8,0 20231107163809 1 20231107152033 2 2...,5w,0 1. d4 d5 2. c4 e6 3. Nc3 Nf6 4. Bg5 Bb4 5...
9,0 20231107163809 1 20231107152033 2 2...,5b,0 1. d4 d5 2. c4 e6 3. Nc3 Nf6 4. Bg5 Bb4 5...


In [147]:
result = test.apply(pgn_parser, axis=1)

In [148]:
combined_df = pd.concat(result.to_list(), ignore_index=True)

In [149]:
combined_df

,id_key,move_number,pgn_string
0,20231107234133,1w,1.e4
1,20231107234133,1b,1.e4 d5
2,20231107234133,2w,1. e4 d5 2.exd5
3,20231107234133,2b,1. e4 d5 2.exd5 Nf6
4,20231107234133,3w,1. e4 d5 2. exd5 Nf6 3.Nc3
...,...,...,...
195,20231107164726,18b,1. e3 d5 2. Nc3 c5 3. Bb5+ Nc6 4. b3 d4 5. Ne4...
196,20231107164726,19w,1. e3 d5 2. Nc3 c5 3. Bb5+ Nc6 4. b3 d4 5. Ne4...
197,20231107164726,19b,1. e3 d5 2. Nc3 c5 3. Bb5+ Nc6 4. b3 d4 5. Ne4...
198,20231107164726,20w,1. e3 d5 2. Nc3 c5 3. Bb5+ Nc6 4. b3 d4 5. Ne4...
